In [2]:
import torch
import pickle
import numpy as np
import os
import json
import sys
sys.path.append('..')


import matplotlib.pyplot as plt

from itertools import product
from hypothesis_generation.hypothesis_utils import HypothesisEval


path_to_json = ("/checkpoint/ramav/adhoc_concept_data/adhoc_images_slurm_v0.2/"
  "hypotheses/v2_typed_simple_fol_depth_6_trials_2000000_ban_1_max_scene_id_200"
      "/5_0.10_hypotheses_heavy.json")
ROOT_DIR = ("/checkpoint/ramav/adhoc_concept_data/adhoc_images_slurm_v0.2/"
  "hypotheses/v2_typed_simple_fol_depth_6_trials_2000000_ban_1_max_scene_id_200")

In [3]:
with open(path_to_json, "rb") as f:
    data = pickle.load(f)
output_fname_prefix = path_to_json.split("hypotheses_heavy")[0].split('/')[-1]

In [160]:
output_fname_prefix

'5_0.10_'

In [4]:
def split_hypotheses(hypotheses, split_a, split_b):
    hyp_a = []
    hyp_b = []
    for hyp in hypotheses:
        if len(set(split_b).intersection(set(hyp.split(" ")))) == 0:
            hyp_a.append(hyp)
        elif len(set(split_a).intersection(set(hyp.split(" ")))) == 0:
            hyp_b.append(hyp)
            
    return hyp_a, hyp_b

def pairwise_splits(hypotheses, split_a, split_b, include_string=""):
    hyp_a = []
    hyp_b = []
    
    for hyp in hypotheses:
        truth_value = []
        for sub in split_b:
            truth_value.append(sub[0] not in hyp.split(" ") or sub[1] not in hyp.split(" ") or include_string not in hyp)
            
        if all(truth_value):
            hyp_a.append(hyp)
        else:
            hyp_b.append(hyp)
        
        #truth_value = []
        #for sub in split_a:
        #    truth_value.append(sub[0] not in hyp.split(" ") or sub[1] not in hyp.split(" "))
        #if all(truth_value):
        #    hyp_b.append(hyp)  
            
    return hyp_a, hyp_b

def gen_pairwise_splits(prop_one, prop_two):
    np.random.seed(42)
    color_materials = np.random.permutation([x for x in product(prop_one, prop_two)])
    total_color_materials = len(color_materials)
    color_split_a = color_materials[:int(0.7 * total_color_materials)]
    color_split_b = color_materials[int(0.7* total_color_materials):]
    
    return color_split_a, color_split_b

def generate_train_val_test_splits(type_substring, split_train, split_val, split_test):
    filename = f"{type_substring}_hypothesis_split_train.json"
    with open(os.path.join(ROOT_DIR, output_fname_prefix + filename), "w") as f:
        json.dump(split_train,f)
        
    filename = f"{type_substring}_hypothesis_split_val.json"
    with open(os.path.join(ROOT_DIR, output_fname_prefix + filename), "w") as f:
        json.dump(split_val, f)
        
    filename = f"{type_substring}_hypothesis_split_test.json"
    with open(os.path.join(ROOT_DIR, output_fname_prefix + filename), "w") as f:
        json.dump(split_test, f)
        
# Generate splits based on length. 



# Generate splits based on function compositions. 



In [6]:
hypothesis_evals = data["hypothesis_and_full_evaluations"]

In [7]:
all_hypotheses = hypothesis_evals.hypothesis

In [169]:
# IID split.
#generate_train_val_test_splits("iid", all_hypotheses, all_hypotheses, all_hypotheses)

In [10]:
# Splits based on color.
color_split_a = ["gray", "red", "blue", "green", "brown"]
color_split_b = ["purple", "cyan", "yellow"]
split_a, split_b = split_hypotheses(all_hypotheses, color_split_a, color_split_b)
print(f"{len(split_a)}, {len(split_b)}")

#generate_train_val_test_splits("color_split", split_a, split_b, split_b)

12152, 2620


In [11]:
split_b[:10]

['2 x locationX? = x color? cyan = and lambda S. for-all=',
 'x color? cyan = cylinder x shape? = and lambda S. for-all=',
 '2 non-x-S color? cyan count= = lambda S. exists=',
 'x locationX? 6 > cyan x color? = and lambda S. exists=',
 'non-x-S color? purple for-all= 5 x locationX? = and lambda S. exists=',
 '8 x locationX? > purple x color? = and lambda S. for-all=',
 'non-x-S locationX? 5 for-all= S color? purple for-all= and lambda S. exists=',
 'S locationY? x locationX? exists= x color? cyan = and lambda S. for-all=',
 'purple x color? = x locationX? 2 > and lambda S. for-all=',
 'yellow x color? = S locationX? x locationY? for-all= and lambda S. exists=']

In [14]:
[x for x in split_a if "color" in x][:10]

['x locationX? x locationY? > S color? blue for-all= and lambda S. exists=',
 'non-x-S color? gray for-all= S shape? cube exists= and lambda S. exists=',
 'S locationX? x locationY? exists= red x color? = and lambda S. for-all=',
 '0.35 x size? = x color? red = and lambda S. for-all=',
 'S shape? cylinder for-all= S color? green for-all= or lambda S.',
 'S color? green for-all= non-x-S size? 0.7 exists= and lambda S. exists=',
 'S color? brown for-all= x size? 0.35 > and lambda S. for-all=',
 'non-x-S locationX? 7 exists= S color? brown exists= and lambda S. exists=',
 'x locationY? 6 > green x color? = and lambda S. exists=',
 'x color? brown = non-x-S locationY? 7 exists= and lambda S. exists=']

In [16]:
# Splits based on shapes
shapes_split_a = ["cube", "sphere"]
shapes_split_b = ["cylinder"]
split_a, split_b = split_hypotheses(all_hypotheses, shapes_split_a, shapes_split_b)
print(f"{len(split_a)}, {len(split_b)}")
#generate_train_val_test_splits("shape_split", split_a, split_b, split_b)

13421, 1484


In [17]:
# Splits based on sizes.
sizes_split_a = ["0.7"]
sizes_split_b = ["0.35"]
split_a, split_b = split_hypotheses(all_hypotheses, sizes_split_a, sizes_split_b)
print(f"{len(split_a)}, {len(split_b)}")
#generate_train_val_test_splits("count_split", split_a, split_b, split_b)

13429, 1480


In [18]:
# Pairwise splits
all_colors = color_split_a + color_split_b
all_materials = material_split_a + material_split_b
all_shapes = shapes_split_a + shapes_split_b
all_sizes = sizes_split_a + sizes_split_b
all_counts = ["1", "2", "3"]
all_locations = ["1", "2", "3", "4", "5", "6", "7", "8"]
all_logical_ops = ["and", "or",]

In [37]:
# Color material split
color_material_split_a, color_material_split_b = gen_pairwise_splits(all_colors, all_materials)
split_a, split_b = pairwise_splits(all_hypotheses, color_material_split_a, color_material_split_b)
print(f"{len(split_a)}, {len(split_b)}")

#generate_train_val_test_splits("color_material_split", split_a, split_b, split_b)

14646, 283


In [38]:
color_material_split_b

array([['green', 'metal'],
       ['purple', 'rubber'],
       ['cyan', 'rubber'],
       ['red', 'metal'],
       ['green', 'rubber']], dtype='<U6')

In [20]:
split_b[:10]

['S material? metal for-all= S color? green for-all= and lambda S.',
 'x material? rubber = non-x-S color? green for-all= and lambda S. exists=',
 'S material? rubber for-all= cyan x color? = and lambda S. exists=',
 'non-x-S color? purple count= S material? rubber count= > lambda S. exists=',
 'x material? rubber = S color? cyan for-all= and lambda S. exists=',
 'S material? rubber for-all= x color? green = and lambda S. exists=',
 'green x color? = S material? metal for-all= and lambda S. for-all=',
 'S material? rubber exists= non-x-S color? green for-all= and lambda S. exists=',
 'S material? metal for-all= x color? green = and lambda S. for-all=',
 'x material? rubber = non-x-S color? purple for-all= and lambda S. exists=']

In [22]:
[x for x in split_a if "material" in x and "color" in x][:10]

['non-x-S color? purple for-all= metal x material? = and lambda S. exists=',
 'rubber x material? = S color? gray for-all= and lambda S. for-all=',
 'S material? rubber exists= S color? red for-all= and lambda S.',
 'metal x material? = non-x-S color? purple for-all= and lambda S. exists=',
 'S material? metal exists= blue x color? = and lambda S. for-all=',
 'gray x color? = S material? metal for-all= and lambda S. exists=',
 'S color? blue for-all= S material? metal exists= and lambda S.',
 'rubber x material? = non-x-S color? gray for-all= and lambda S. exists=',
 'non-x-S color? red for-all= rubber x material? = and lambda S. exists=',
 'S color? yellow for-all= rubber x material? = and lambda S. exists=']

In [27]:
# Color count split
color_count_split_a, color_count_split_b = gen_pairwise_splits(all_counts, all_colors)
split_a, split_b = pairwise_splits(all_hypotheses, color_count_split_a, color_count_split_b, include_string="count=")
print(f"{len(split_a)}, {len(split_b)}")
#generate_train_val_test_splits("color_count_split", split_a, split_b, split_b)

14882, 47


In [31]:
[x for x in split_a if "color" in x]

['x locationX? x locationY? > S color? blue for-all= and lambda S. exists=',
 '2 x locationX? = x color? cyan = and lambda S. for-all=',
 'non-x-S color? gray for-all= S shape? cube exists= and lambda S. exists=',
 'S locationX? x locationY? exists= red x color? = and lambda S. for-all=',
 '0.35 x size? = x color? red = and lambda S. for-all=',
 'x color? cyan = cylinder x shape? = and lambda S. for-all=',
 'S shape? cylinder for-all= S color? green for-all= or lambda S.',
 'non-x-S color? cyan for-all= x color? gray = and lambda S. exists=',
 'x locationX? 6 > cyan x color? = and lambda S. exists=',
 'S color? green for-all= non-x-S size? 0.7 exists= and lambda S. exists=',
 'S color? brown for-all= x size? 0.35 > and lambda S. for-all=',
 'non-x-S color? purple for-all= 5 x locationX? = and lambda S. exists=',
 'non-x-S locationX? 7 exists= S color? brown exists= and lambda S. exists=',
 'x locationY? 6 > green x color? = and lambda S. exists=',
 'x color? brown = non-x-S locationY? 

In [35]:
# Color location split
color_loc_split_a, color_loc_split_b = gen_pairwise_splits(all_locations, all_colors)
split_a, split_b = pairwise_splits(all_hypotheses, color_loc_split_a, color_loc_split_b, include_string="location")
print(f"{len(split_a)}, {len(split_b)}")
#generate_train_val_test_splits("color_location_split", split_a, split_b, split_b)

14179, 750


In [36]:
color_loc_split_b

array([['7', 'gray'],
       ['1', 'red'],
       ['3', 'purple'],
       ['1', 'blue'],
       ['8', 'cyan'],
       ['5', 'yellow'],
       ['5', 'green'],
       ['3', 'yellow'],
       ['7', 'purple'],
       ['2', 'blue'],
       ['3', 'cyan'],
       ['3', 'blue'],
       ['8', 'yellow'],
       ['3', 'brown'],
       ['1', 'yellow'],
       ['6', 'blue'],
       ['2', 'cyan'],
       ['4', 'brown'],
       ['7', 'green'],
       ['5', 'cyan']], dtype='<U6')

In [26]:
print("Test")
print(split_b[:10])

print("Train")
print(split_a[:10])

Test
['2 x locationX? = x color? cyan = and lambda S. for-all=', 'S color? red exists= x locationY? 1 = and lambda S. exists=', 'non-x-S locationY? 6 exists= x color? blue = and lambda S. exists=', 'purple x color? = S locationX? 3 for-all= and lambda S. exists=', 'S color? green for-all= 7 x locationX? > and lambda S. exists=', 'x locationX? 3 = S color? yellow for-all= and lambda S. for-all=', 'S color? cyan for-all= x locationX? 5 > or lambda S. for-all=', 'x locationY? 1 = x color? red = and lambda S. exists=', 'S locationX? 2 exists= S color? cyan for-all= and lambda S.', 'purple x color? = S locationY? 3 for-all= and lambda S. for-all=']
Train
['x locationX? x locationY? > S color? blue for-all= and lambda S. exists=', 'non-x-S locationX? 5 for-all= x locationY? 7 = and lambda S. exists=', 'non-x-S color? gray for-all= S shape? cube exists= and lambda S. exists=', 'S locationX? x locationY? exists= red x color? = and lambda S. for-all=', 'S shape? cube exists= non-x-S locationX? 

In [33]:
# Color boolean split
color_bool_split_a, color_bool_split_b = gen_pairwise_splits(all_colors, all_logical_ops)
split_a, split_b = pairwise_splits(all_hypotheses, color_bool_split_a, color_bool_split_b)
print(f"{len(split_a)}, {len(split_b)}")

#generate_train_val_test_splits("color_boolean_split", split_a, split_b, split_b)

12335, 2594


In [34]:
color_bool_split_b

array([['green', 'or'],
       ['purple', 'and'],
       ['cyan', 'and'],
       ['red', 'or'],
       ['green', 'and']], dtype='<U6')

In [8]:
# Length based split
all_lengths = [len(x.split(' ')) for x in all_hypotheses]
split_a = [x for x in all_hypotheses if len(x.split(' ')) < 10]
split_b = [x for x in all_hypotheses if len(x.split(' ')) >= 10]
print(f"{len(split_a)}, {len(split_b)}")

#generate_train_val_test_splits("length_threshold_10_split", split_a, split_b, split_b)
generate_train_val_test_splits("inferential_coherence", split_b, split_a, split_a)

408, 14521


In [9]:
# Splits based on material.
material_split_a = ["rubber"]
material_split_b = ["metal"]
split_a, split_b = split_hypotheses(all_hypotheses, material_split_a, material_split_b)
print(f"{len(split_a)}, {len(split_b)}")

#generate_train_val_test_splits("material_split", split_a, split_b, split_b)

13476, 1453
